In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from src.ft_math import mean, softmax, sigmoid, cross_entropy

In [8]:
df = pd.read_csv('./data/data.csv')

In [9]:
df

,id,diagnosis,radius,texture,perimeter,area,compactness,smoothness,concavity,concave_point,...,radiusW,textureW,perimeterW,areaW,compactnessW,smoothnessW,concavityW,concave_pointsW,symmetryW,fractal_dimensionW
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [34]:
# %load './src/Perceptron.py'
from src.ft_math import relu, softmax
from src.ft_math import cross_entropy as cost
import numpy as np
import sklearn.preprocessing


# https://towardsdatascience.com/building-neural-network-from-scratch-9c88535bf8e9

class Layer:

    # A building block. Each layer is capable of performing two things:
    # - Process input to get output:
    """ output = layer.forward(input) """
    # - Propagate gradients through itself:
    """ grad_input = layer.backward(input, grad_output) """
    # Some layers also have learnable parameters which they update during
    # layer.backward.

    def __init__(self):
        # Here we can initialize layer parameters (if any) and auxiliary stuff.
        # A dummy layer does nothing
        pass

    def forward(self, input):
        # Takes input data of shape [batch, input_units],
        # returns output data [batch, output_units]
        # A dummy layer just returns whatever it gets as input.
        return input

    def backward(self, input, grad_output):
        # Performs a backpropagation step through the layer, with respect to
        # the given input.
        # To compute loss gradients w.r.t input, we need to apply chain
        # rule (backprop):
        """ d loss / d x  = (d loss / d layer) * (d layer / d x) """
        # Luckily, we already receive d loss / d layer as input, so you only
        # need to multiply it by d layer / d x.
        # If our layer has parameters (e.g. dense layer), we need to update
        # them here using d loss / d layer
        # The gradient of a dummy layer is just grad_output, but we'll write
        # more explicitly
        num_units = input.shape[1]
        d_layer_d_input = np.eye(num_units)
        return np.dot(grad_output, d_layer_d_input)  # chain rule


class ReLU(Layer):

    def __init__(self):
        pass

    def forward(self, input):
        relu_forward = np.maximum(0,input)
        return relu_forward

    def backward(self, input, grad_output):
        relu_grad = input > 0
        return grad_output * relu_grad


class Dense(Layer):

    def __init__(self, input_units, output_units, learning_rate=0.1):
        """ A dense layer performs a learned affine transformation:
            f(x) = <W*x> + b """
        self.learning_rate = learning_rate
        self.weights = np.random.normal(
            loc=0.0,
            scale=np.sqrt(2/(input_units + output_units)),
            size=(input_units, output_units)
            )
        self.biases = np.zeros(output_units)

    def forward(self, input):
        """ Perform an affine transformation:
            f(x) = <W*x> + b """

        # input shape: [batch, input_units]
        # output shape: [batch, output units]

        return np.dot(input.T, self.weights) + self.biases

    def backward(self, input, grad_output):
        # compute d f / d x = d f / d dense * d dense / d x
        # where d dense/ d x = weights transposed
        grad_input = np.dot(grad_output, self.weights.T)

        # compute gradient w.r.t. weights and biases
        grad_weights = np.dot(input, grad_output)
        grad_biases = grad_output.mean(axis=0) * input.shape[0]

        assert grad_weights.shape == self.weights.shape
        assert grad_biases.shape == self.biases.shape

        # Here we perform a stochastic gradient descent step to update the weights
        self.weights = self.weights - self.learning_rate * grad_weights
        self.biases = self.biases - self.learning_rate * grad_biases

        return grad_input


def softmax_crossentropy_with_logits(logits, reference_answers):
    logits_for_answers = logits[np.arange(len(logits)), reference_answers]
    xentropy = - logits_for_answers + np.log(np.sum(np.exp(logits), axis=-1))
    return xentropy


def grad_softmax_crossentropy_with_logits(logits, reference_answers):
    ones_for_answers = np.zeros_like(logits)
    ones_for_answers[np.arange(len(logits)), reference_answers] = 1

    softmax = np.exp(logits) / np.exp(logits).sum(axis=-1, keepdims=True)

    return (- ones_for_answers + softmax) / logits.shape[0]


if __name__ == "__main__":
    """ n = 8
    l0 = Dense(n, 2)
    X = np.eye(n)
    print(l0.forward(X))
    print(l0.backward(X, l0.forward(X)))
    print() """
    x2 = np.array([1, 1, 1, 1, 1, 1])
    x3 = x2


In [35]:
X_train = normalize(np.array(df.iloc[:,2:22])).T
X_train = X_train.reshape([X_train.shape[0], -1])
y_train = np.array(df['diagnosis'].map({'M':1,'B':0}))

In [36]:
network = []
network.append(Dense(X_train.shape[0], 20))
network.append(ReLU())
network.append(Dense(20, 10))
network.append(ReLU())
network.append(Dense(10, 2))

In [37]:
def forward(network, X):
    activations = []
    input = X
    for l in network:
        activations.append(l.forward(input))
        input = activations[-1]
    assert len(activations) == len(network)
    return activations

In [38]:
def predict(network, X):
    logits = forward(network, X)[-1]
    return logits.argmax(axis=1)  # the index of the most probable value

In [39]:
def train(network, X, y):
    layer_activations = forward(network, X)
    layer_inputs = [X] + layer_activations
    logits = layer_activations[-1]
    loss = softmax_crossentropy_with_logits(logits,y)
    loss_grad = grad_softmax_crossentropy_with_logits(logits,y)
    for layer_index in range(len(network))[::-1]:
        layer = network[layer_index]
        loss_grad = layer.backward(layer_inputs[layer_index], loss_grad)
    return np.mean(loss)

In [40]:
from tqdm import trange
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [45]:
from IPython.display import clear_output
train_log = []
val_log = []
for epoch in range(25):
    print(f'xtrain.shape:{X_train.shape},y_train.shape{y_train.shape}')
    for x_batch,y_batch in iterate_minibatches(X_train.T,y_train,batchsize=10,shuffle=True):
        train(network,x_batch,y_batch)
    
    train_log.append(np.mean(predict(network,X_train)==y_train))
    val_log.append(np.mean(predict(network,X_val)==y_val))
    
    clear_output()
    print("Epoch",epoch)
    print("Train accuracy:",train_log[-1])
    print("Val accuracy:",val_log[-1])
    plt.plot(train_log,label='train accuracy')
    plt.plot(val_log,label='val accuracy')
    plt.legend(loc='best')
    plt.grid()
    plt.show()

  0%|          | 0/56 [00:00<?, ?it/s]

xtrain.shape:(20, 569),y_train.shape(569,)


ValueError: shapes (20,10) and (20,20) not aligned: 10 (dim 1) != 20 (dim 0)